In [1]:
import multiprocessing as mp

In [3]:
import numpy as np
from time import time

In [13]:
# Prepare data
# value range
r = 10
# number of rows
# m = 1000
m = 100000
# number of columns
# n = 10000
n = 10

np.random.seed(100)
arr = np.random.randint(0, r, size=[m, n])
data = arr.tolist()
print(data[:10])

[[8, 8, 3, 7, 7, 0, 4, 2, 5, 2], [2, 2, 1, 0, 8, 4, 0, 9, 6, 2], [4, 1, 5, 3, 4, 4, 3, 7, 1, 1], [7, 7, 0, 2, 9, 9, 3, 2, 5, 8], [1, 0, 7, 6, 2, 0, 8, 2, 5, 1], [8, 1, 5, 4, 2, 8, 3, 5, 0, 9], [3, 6, 3, 4, 7, 6, 3, 9, 0, 4], [4, 5, 7, 6, 6, 2, 4, 2, 7, 1], [6, 6, 0, 7, 2, 3, 5, 4, 2, 4], [3, 7, 9, 0, 0, 5, 9, 6, 6, 5]]


In [15]:

# Sequential code: Solution Without Parallelization

def howmany_within_range(row, minimum, maximum):
    """Returns how many numbers lie within `maximum` and `minimum` in a given `row`"""
    #print(mp.current_process(),' ',row)
    count = 0
    for num in row:
       if minimum <= num <= maximum:
          count = count + 1
    return count

# begin timing
start_time = time()

results = []
for row in data:
   results.append(howmany_within_range(row, minimum=4, maximum=8))

# end timing
print(f'Total time: {time()-start_time} seconds')

print(results[:10])


Total time: 0.0425570011138916 seconds
[6, 3, 5, 4, 4, 5, 5, 7, 6, 5]


In [7]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 23.10
Release:	23.10
Codename:	mantic


In [8]:
!uname -a

Linux joaom-Type1ProductConfigId 6.5.0-17-generic #17-Ubuntu SMP PREEMPT_DYNAMIC Thu Jan 11 14:01:59 UTC 2024 x86_64 x86_64 x86_64 GNU/Linux


In [9]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 140
model name	: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz
stepping	: 1
microcode	: 0xb4
cpu MHz		: 2779.636
cache size	: 12288 KB
physical id	: 0
siblings	: 8
core id		: 0
cpu cores	: 4
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 27
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc art arch_perfmon pebs bts rep_good nopl xtopology nonstop_tsc cpuid aperfmperf tsc_known_freq pni pclmulqdq dtes64 monitor ds_cpl vmx est tm2 ssse3 sdbg fma cx16 xtpr pdcm pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand lahf_lm abm 3dnowprefetch cpuid_fault epb cat_l2 invpcid_single cdp_l2 ssbd ibrs ibpb stibp ibrs_enhanced tpr_shadow flexpriority ept vpid ept_ad fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid rdt_a avx512f avx512dq rdseed 

In [17]:
# Option 1 pool apply


num_cpus = mp.cpu_count()
print('Num cpus = ', num_cpus)

start_time = time()

pool = mp.Pool(mp.cpu_count())
print('Time to create pool: ',round(time() - start_time,8), 'seconds')

results = [pool.apply(howmany_within_range, args=(row, 4, 8)) for row in data]

pool.close()

print(f'Total time: {time()-start_time} seconds')

print(results[:10])




Num cpus =  8
Time to create pool:  0.01780224 seconds
Total time: 6.619020700454712 seconds
[6, 3, 5, 4, 4, 5, 5, 7, 6, 5]


In [18]:
# Option 2 pool map


def howmany_within_range_rowonly(row, minimum=4, maximum=8):
  # print(mp.current_process(),' ',row) # this will print the process object and the item it is working with
  count = 0
  for num in row:
     if minimum <= num <= maximum:
        count = count + 1
  return count

start_time = time()

pool = mp.Pool(mp.cpu_count())
results = pool.map(howmany_within_range_rowonly, [row for row in data])

pool.close()

print(f'Total time: {time()-start_time} seconds')

print(results[:10])



Total time: 0.054193973541259766 seconds
[6, 3, 5, 4, 4, 5, 5, 7, 6, 5]


In [ ]:
# Option 3 pool starmap